In [1]:
# #@title Installing libraries { display-mode: "form" }
# !pip3 install transformers 
# !pip3 install tensorboard 
# !pip3 install simpletransformers
# !pip3 install jsonlines
# !pip3 install pytelegrambotapi
# !pip install tqdm

In [3]:
#@title Setup & Config & Imports
# !mkdir /usr/local/lib/python3.6/dist-packages/tqdm-4.41.1.dist-info/METADATA

from tqdm.notebook import tqdm

# class _TQDM(tqdm.tqdm):
#     def __init__(self, *argv, **kwargs):
#         kwargs['disable'] = True
#         if kwargs.get('disable_override', 'def') != 'def':
#             kwargs['disable'] = kwargs['disable_override']
#         super().__init__(*argv, **kwargs)
# tqdm.tqdm = _TQDM
 
import pandas as pd
import numpy as np
import json
import jsonlines
import re
from collections import defaultdict, Counter
 
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['text.color'] = mpl.rcParams['axes.labelcolor'] = mpl.rcParams['xtick.color'] = mpl.rcParams['ytick.color'] = 'white'
 
%pylab inline
from sklearn.metrics import *
 
try:
    from simpletransformers.classification import ClassificationModel, ClassificationArgs
except (FileNotFoundError, AttributeError):
    from simpletransformers.classification import ClassificationModel, ClassificationArgs
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
 
# %load_ext tensorboard
 
import string
 
RS = 179
PUNCTUATION = string.punctuation
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager

mkdir: cannot create directory ‘/usr/local/lib/python3.6/dist-packages/tqdm-4.41.1.dist-info/METADATA’: No such file or directory
Populating the interactive namespace from numpy and matplotlib


In [1]:
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager

Building jupyterlab assets (build:prod:minimize)


In [4]:
# !rm -rf sample_data/

In [2]:
import time

for _ in tqdm(range(100)):
    time.sleep(1)

NameError: name 'tqdm' is not defined

# Downloading data

In [5]:
# !curl https://onti2020.ai-academy.ru/task/rucos.zip -o data.zip
# !unzip -o data.zip

# !mv RuCoS/ data/

In [6]:
!ls -lah data/

total 211M
drwxr-xr-x  2 jovyan jovyan 4.0K Feb 26 12:40 .
drwxrwxrwx 20 root   root   4.0K Mar  2 11:45 ..
-rw-r--r--  1 jovyan jovyan 6.1K Nov 10 21:06 .DS_Store
-rwxr-xr-x  1 jovyan jovyan  17M Oct 15 03:28 rucos_test.jsonl
-rwxr-xr-x  1 jovyan jovyan 175M Oct 15 03:26 rucos_train.jsonl
-rwxr-xr-x  1 jovyan jovyan  19M Oct 15 03:25 rucos_val.jsonl


# Process data

In [28]:
class QADataset: 
    """Question and answers dataset""" 
     
    def __init__(self, path, making_smaller=False, lim=-1): 
        """ 
        Args: 
            path (string): Path to jsonl file 
        """ 
        self.path = path 
        data = [] 
        self.text = [] 
        with open(path, 'r') as json_file: 
            json_list = list(json_file) 
            for json_str in tqdm(json_list if lim == -1 else json_list[:lim]): 
                item = json.loads(json_str) 
                text = item['passage']['text'] 
                self.text.append(text) 
                entities = item['passage']['entities'] 
                used = set()
                for en in entities:
                    word = text[en['start']:en['end']] 
                    if word not in used:
                        used.add(word)
                entities = list(used)

                for row in item['qas']: 
                    for ans in row.get('answers', [None]): 
                        q_text = row['query'] 
                        for i, word in enumerate(entities): 
                            n_q = q_text.replace('@placeholder', word) 
                            data.append({"idx": f'{item["idx"]}/{i}', 
                                         "text_id": len(self.text) - 1,
                                         "query": n_q})
                            
                            if not ans:
                                continue 
 
                            label = int(ans['text'].strip() == word.strip())
                            if not making_smaller or label or random.randint(1, 10) == 1:
                                data[-1]["label"] = label
                            else:
                                data.pop()
                self.text[-1] = re.sub(r'(@\w+)|(\\n)', " ", self.text[-1])
        self.data = pd.DataFrame.from_dict(data).set_index("idx")
     
    def __len__(self): 
        return len(self.data) 
 
    def __getitem__(self, ind):  
        if type(ind) == int: 
            row = self.data.iloc[ind] 
            text_a = self.text[row.text_id].split() 
            text_b = row.query 
 
            w = 512 - len(text_b.split()) - 5 
            e = random.randint(0, len(text_a) - w) 
            text_a = ' '.join(text_a[e:e + w]) 
 
            res = [[list(self.data.index)[ind], text_a, text_b, row.label if 'label' in self.data.columns else None]] 
            return pd.DataFrame(res, columns=["idx", "text_a", "text_b", "labels"]).set_index("idx") 
        if type(ind) == slice: 
            res = [] 
            rows = self.data.iloc[ind] 
            for idx, row in tqdm(rows.iterrows(), total=len(rows)): 
                text_a = self.text[row.text_id].split() 
                text_b = row.query 
 
                w = 512 - len(text_b.split()) - 5 
                e = random.randint(0, max(1, len(text_a) - w)) 
                text_a = ' '.join(text_a[e:e + w]) 
 
                res.append([idx, text_a, text_b, row.label if 'label' in self.data.columns else None]) 
            return pd.DataFrame(res, columns=["idx", "text_a", "text_b", "labels"]).set_index("idx")
 
 
    def get_df(self): 
        texts = self.data['data'] 
        labels = self.data['label'] if 'label' in self.data.columns else [None] * self.data.shape[0] 
 
        return pd.DataFrame({ 
                    'text': texts, 
                    'labels': labels 
                })

In [29]:
train_df = QADataset('data/rucos_train.jsonl', making_smaller=True)
val_df = QADataset('data/rucos_val.jsonl')
test_df = QADataset('data/rucos_test.jsonl')

  0%|          | 0/72193 [00:00<?, ?it/s]

  0%|          | 0/7577 [00:00<?, ?it/s]

  0%|          | 0/7257 [00:00<?, ?it/s]

In [9]:
train_df[:10]

100%|██████████| 10/10 [00:00<00:00, 5927.51it/s]


,text_a,text_b,labels
idx,,,
0/3,"Наблюдатели полагают, что подоплекой теракта в...","Кроме того, серьезным вызовом для России стано...",1
1/10,О вторжении на Украину танковой колонны из РФ ...,Россия категорически опровергла сообщение СНБО...,0
1/11,О вторжении на Украину танковой колонны из РФ ...,Россия категорически опровергла сообщение Лысе...,1
1/4,О вторжении на Украину танковой колонны из РФ ...,Россия категорически опровергла сообщение Андр...,1
1/7,О вторжении на Украину танковой колонны из РФ ...,Россия категорически опровергла сообщение ОПЕК...,0
2/0,Год назад Владимир Путин вновь стал президенто...,"Кремле, руководитель отдела Восточной Европы и...",0
2/2,Год назад Владимир Путин вновь стал президенто...,"Инго Маннтойфель, руководитель отдела Восточно...",1
2/4,Год назад Владимир Путин вновь стал президенто...,"Россия, руководитель отдела Восточной Европы и...",0
2/5,Год назад Владимир Путин вновь стал президенто...,"Германии, руководитель отдела Восточной Европы...",0


# Model params

In [10]:
# @title Model params { run: "auto" }
model_name = "DeepPavlov/rubert-base-cased" #@param {type:"string"}
num_train_epochs = 2 #@param {type:"slider", min:1, max:50, step:1}
use_cuda = True #@param ["True", "False"] {type:"raw"}
learning_rate =  5e-5 #@param {type:"number"}
train_batch_size = 32 #@param {type:"slider", min:1, max:100, step:1}
no_save = True #@param ["True", "False"] {type:"raw"}
adam_epsilon =  3e-8 #@param {type:"number"}

# Creating model

In [11]:
model = ClassificationModel(model_type='bert',
                            model_name=model_name,
                            use_cuda=use_cuda,
                            num_labels=2,

                            weight=list(dict(Counter(train_df[:].labels)).values())[::-1],

                            args=ClassificationArgs(
                                                    num_train_epochs=num_train_epochs,
                                                    learning_rate=learning_rate,
                                                    adam_epsilon=adam_epsilon,
                                                    
                                                    train_batch_size=train_batch_size,
                                                    max_seq_length=512,
                                                    manual_seed=RS,
                                                    
                                                    overwrite_output_dir=True,
                                                    no_cache=False,
                                                    no_save=no_save,
                                                    save_eval_checkpoints=False,
                                                    save_model_every_epoch=False,
                                                    save_steps=-1,
                                            )
                            )

100%|██████████| 384053/384053 [00:39<00:00, 9746.32it/s] 


Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/711M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

# Training

In [12]:
model.train_model(train_df[:])

100%|██████████| 384053/384053 [00:40<00:00, 9529.36it/s] 


  0%|          | 0/189 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/12002 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/12002 [00:00<?, ?it/s]

(24004, 0.3047286289675173)

# Predict

In [22]:
def save_pred(filename, preds):
    with open('data/rucos_test.jsonl', 'r') as json_file:
        test = list(map(lambda x: json.loads(x), list(json_file)))
    ind = 0
    res = []
    
    for i in range(len(test)):
        ent_prob = []
        
        entities = test[i]['passage']['entities']
        used = set()
        for en in entities:
            word = test[i]['passage']['text'][en['start']:en['end']] 
            if word not in used:
                used.add(word)
        entities = list(used)
        
        for j in range(len(entities)):
            ent_prob.append(preds[ind])
            ind += 1
        ent_prob = np.array(ent_prob).argmax()
        
#         start, end = test[i]['passage']["entities"][ent_prob]['start'], test[i]['passage']["entities"][ent_prob]['end']
        answer = entities[ent_prob]
        res.append({"idx": test[i]["idx"], "text": answer})
    
    !mkdir submitions/
    
    with jsonlines.open("submitions/" + filename, 'w') as file:
        file.write_all(res)

In [14]:
predictions, raw_outputs = model.predict(list(zip(test_df[:].text_a, test_df[:].text_b)))

100%|██████████| 67751/67751 [00:06<00:00, 10482.21it/s]


  0%|          | 0/8469 [00:00<?, ?it/s]

In [19]:
cnt = Counter(predictions)

print(f'    cnt: {cnt}')
print(f"0 label: {cnt[0] / len(predictions) * 100:.3f}%")
print(f"1 label: {cnt[1] / len(predictions) * 100:.3f}%")

    cnt: Counter({0: 51927, 1: 15824})
0 label: 76.644%
1 label: 23.356%


In [20]:
from scipy.special import softmax
pred_proba = np.array([elem[1] for elem in softmax(raw_outputs, axis=1)])
pred_proba

array([4.61905833e-04, 2.62553008e-02, 4.57419041e-04, ...,
       7.49270719e-01, 6.47519657e-01, 1.02529141e-01])

In [23]:
file_name = f'{__import__("datetime").datetime.now()}.jsonl'
save_pred(file_name, pred_proba)

mkdir: cannot create directory ‘submitions/’: File exists


In [25]:
#@title Telegram Logger (doc sender) { run: "auto", vertical-output: true, display-mode: "form" }
token = "1282076787:AAEC3rYsSHh_livmutl0qDEemVhlhjWpWyI" #@param {type:"string"}
user_id =  -584355986 #@param {type:"integer"}
combine = True #@param ["True", "False"] {type:"raw"}

import telebot
bot = telebot.TeleBot(token)
last_message = None

def logger(txt):
    global last_message
    while True:
        try:
            last_message_text = last_message.text if last_message != None else ''

            if type(txt) is tqdm:
                txt = txt.format_dict

                rate = txt["rate"] if txt["rate"] else 0
                remain = int((txt['total'] - txt['n']) / rate) if rate != 0 else 0

                txt = f"{txt['prefix']} - {int(txt['n'] / txt['total'] * 100)}%, {txt['n']}/{txt['total']} [{str(int(txt['elapsed']) // 60).rjust(2, '0')}:{str(int(txt['elapsed']) % 60).rjust(2, '0')}<{str(remain // 60).rjust(2, '0')}:{str(remain % 60).rjust(2, '0')}]"
                
                last_message_text = '\n'.join(last_message_text.split('\n')[:-1])

            if not last_message or not combine:
                last_message = bot.send_message(user_id, txt)
            else:
                last_message = bot.edit_message_text(chat_id=user_id,
                                      message_id=last_message.message_id,
                                      text=last_message_text + '\n' + txt)
            return
        except Exception as ex:
            print("Exception while logging:")
            print(ex)
            pass

In [26]:
while True:
    try:
        bot.send_document(user_id, 
                          open('submitions/' + file_name, 'rb'),
                          caption=f'Model: {model_name}\nLR: {learning_rate}\nAdam eps: {adam_epsilon}\nEpochs: {num_train_epochs}\n{cnt}\nLabel 0: {cnt[0] / len(predictions) * 100:.3f}%\nLabel 1: {cnt[1] / len(predictions) * 100:.3f}%')
        break
    except:
        import time
        time.sleep(1)